In [6]:
import pandas as pd
import numpy as np

import json

import datetime as dt

import os

In [7]:
df_station_locations = pd.read_csv(r"C:\Users\mkaya\OneDrive\Masaüstü\istanbul112_hidden\data\locations\station_locations.csv")


In [8]:
df_station_locations.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True)

In [9]:
df_station_locations

,Ekip Adı,lat,lon,Bölge,city,district,station_count,rural,double_station,double_station_with,Neighbourhood,District
0,ARN1,41.177852,28.745802,ARN,ARNAVUTKÖY,ARNAVUTKÖY MERKEZ,1,KENTSEL,False,NaN,Arnavutköy Merkez Mahallesi,Arnavutköy
1,ARN2,41.156037,28.623824,ARN,ARNAVUTKÖY,HADIMKÖY,1,KENTSEL,False,NaN,Hadımköy Mahallesi,Arnavutköy
2,ARN3,41.204411,28.716732,ARN,ARNAVUTKÖY,MAREŞAL FEVZİ ÇAKMAK,1,KENTSEL,False,NaN,Mareşal Fevzi Çakmak Mahallesi,Arnavutköy
3,ARN4,41.199450,28.760639,ARN,ARNAVUTKÖY,HİCRET,1,KENTSEL,False,NaN,Hicret Mahallesi,Arnavutköy
4,ARN6,41.185858,28.708606,ARN,ARNAVUTKÖY,TAŞOLUK,1,KENTSEL,False,NaN,Taşoluk Mahallesi,Arnavutköy
...,...,...,...,...,...,...,...,...,...,...,...,...
207,ZTB7,41.006756,28.897603,ZTB,ZEYTİNBURNU,SEYİTNİZAM,1,KENTSEL,False,NaN,Seyitnizam Mahallesi,Zeytinburnu
208,ARN5,41.265160,28.688026,ARN,ARNAVUTKÖY,TAYAKADIN,1,KIRSAL,False,NaN,Tayakadın Mahallesi,Arnavutköy
209,ÇTL4,41.413472,28.186711,ÇTL,ÇATALCA,FATİH,1,KIRSAL,False,NaN,Fatih Mahallesi,Çatalca
210,EYP6,41.253788,28.714775,EYP,ARNAVUTKÖY,TAYAKADIN,1,KIRSAL,True,EYP7,Tayakadın Mahallesi,Arnavutköy


In [10]:
example_json_input = '''
    {
        "stations": [
            {
                "id": "BYR1",
                "region": "Beyoğlu",
                "importance": 5,
                "assignedPersonnel": [
                    {"id": "P001", "name": "Ahmet Yılmaz", "roles": ["medic"], "assignedFrom": "BYR1", "homeStationId": "BYR1", "negativeStations": ["BYR2"], "preferredStations": ["BYR1", "SIS1"], "rotationCount": 0},
                    {"id": "P002", "name": "Mehmet Kaya", "roles": ["medic"], "assignedFrom": "BYR1", "homeStationId": "BYR1", "negativeStations": [], "preferredStations": ["BYR1", "BYR3"], "rotationCount": 1},
                    {"id": "P003", "name": "Ali Demir", "roles": ["driver"], "assignedFrom": "BYR1", "homeStationId": "BYR1", "negativeStations": [], "preferredStations": [], "rotationCount": 0}
                ],
                "maxRotationCount": 2
            },
            {
                "id": "BYR2",
                "region": "Beyoğlu",
                "importance": 3,
                "assignedPersonnel": [
                    {"id": "P004", "name": "Ayşe Öztürk", "roles": ["medic"], "assignedFrom": "BYR2", "homeStationId": "BYR2", "negativeStations": [], "preferredStations": ["BYR2", "FTH1"], "rotationCount": 0},
                    {"id": "P005", "name": "Fatma Şahin", "roles": ["medic"], "assignedFrom": "BYR2", "homeStationId": "BYR2", "negativeStations": [], "preferredStations": ["BYR2"], "rotationCount": 2}
                ],
                "maxRotationCount": 3
            },
            {
                "id": "SIS1",
                "region": "Şişli",
                "importance": 4,
                "assignedPersonnel": [
                    {"id": "P006", "name": "Mustafa Çelik", "roles": ["medic", "driver"], "assignedFrom": "SIS1", "homeStationId": "SIS1", "negativeStations": ["BYR2"], "preferredStations": ["SIS1", "SIS2"], "rotationCount": 1},
                    {"id": "P007", "name": "Zeynep Yıldız", "roles": ["medic"], "assignedFrom": "SIS1", "homeStationId": "SIS1", "negativeStations": [], "preferredStations": ["SIS1", "BYR1"], "rotationCount": 0},
                    {"id": "P008", "name": "Hasan Arslan", "roles": ["driver"], "assignedFrom": "SIS1", "homeStationId": "SIS1", "negativeStations": [], "preferredStations": [], "rotationCount": 3},
                    {"id": "P009", "name": "Osman Kara", "roles": ["medic"], "assignedFrom": "SIS1", "homeStationId": "SIS1", "negativeStations": [], "preferredStations": ["SIS1", "FTH1"], "rotationCount": 0}
                ],
                "maxRotationCount": 4
            },
            {
                "id": "FTH1",
                "region": "Fatih",
                "importance": 5,
                "assignedPersonnel": [
                    {"id": "P010", "name": "Emine Aydın", "roles": ["medic"], "assignedFrom": "FTH1", "homeStationId": "FTH1", "negativeStations": [], "preferredStations": ["FTH1"], "rotationCount": 1}
                ],
                "maxRotationCount": 2
            }
        ],
        neighboringRegions = {
            "Adalar": ["Kartal", "Pendik"],
            "Arnavutköy": ["Çatalca", "Esenler", "Başakşehir", "Eyüpsultan"],
            "Ataşehir": ["Üsküdar", "Kadıköy", "Maltepe", "Ümraniye"],
            "Avcılar": ["Küçükçekmece", "Bağcılar", "Başakşehir"],
            "Bağcılar": ["Küçükçekmece", "Bahçelievler", "Güngören", "Esenler", "Avcılar"],
            "Bahçelievler": ["Bağcılar", "Bakırköy", "Güngören"],
            "Bakırköy": ["Bahçelievler", "Güngören", "Küçükçekmece", "Esenyurt"],
            "Başakşehir": ["Arnavutköy", "Eyüpsultan", "Esenler", "Avcılar", "Küçükçekmece"],
            "Bayrampaşa": ["Gaziosmanpaşa", "Eyüpsultan", "Fatih", "Şişli"],
            "Beşiktaş": ["Şişli", "Beyoğlu", "Sarıyer", "Kağıthane"],
            "Beykoz": ["Ümraniye", "Çekmeköy", "Sancaktepe", "Üsküdar"],
            "Beylikdüzü": ["Büyükçekmece", "Esenyurt", "Avcılar"],
            "Beyoğlu": ["Şişli", "Kağıthane", "Fatih", "Beşiktaş", "Eyüpsultan"],
            "Büyükçekmece": ["Çatalca", "Silivri", "Esenyurt", "Beylikdüzü"],
            "Çatalca": ["Silivri", "Büyükçekmece", "Arnavutköy"],
            "Çekmeköy": ["Ümraniye", "Beykoz", "Üsküdar", "Sancaktepe"],
            "Esenler": ["Bağcılar", "Güngören", "Bakırköy", "Başakşehir", "Gaziosmanpaşa"],
            "Esenyurt": ["Küçükçekmece", "Avcılar", "Bakırköy", "Beylikdüzü", "Büyükçekmece"],
            "Eyüpsultan": ["Sarıyer", "Kağıthane", "Beyoğlu", "Gaziosmanpaşa", "Bayrampaşa", "Fatih", "Sultangazi", "Başakşehir", "Arnavutköy"],
            "Fatih": ["Beyoğlu", "Şişli", "Eminönü (merged into Fatih)", "Esenler?"],
            "Gaziosmanpaşa": ["Eyüpsultan", "Sultangazi", "Esenler", "Bayrampaşa"],
            "Güngören": ["Bağcılar", "Bahçelievler", "Bakırköy", "Esenler", "Zeytinburnu"],
            "Kadıköy": ["Ümraniye", "Ataşehir", "Maltepe", "Kartal", "Sancaktepe"],
            "Kağıthane": ["Şişli", "Beşiktaş", "Beyoğlu", "Eyüpsultan", "Sarıyer", "Sultangazi"],
            "Kartal": ["Maltepe", "Pendik", "Tuzla", "Adalar"],
            "Küçükçekmece": ["Avcılar", "Bağcılar", "Bakırköy", "Esenyurt", "Başakşehir"],
            "Maltepe": ["Kadıköy", "Kartal", "Pendik", "Sancaktepe", "Ümraniye"],
            "Pendik": ["Kartal", "Tuzla", "Sancaktepe", "Maltepe", "Ümraniye"],
            "Sancaktepe": ["Çekmeköy", "Ümraniye", "Kadıköy", "Maltepe", "Pendik", "Tuzla", "Kartal"],
            "Sarıyer": ["Eyüpsultan", "Kağıthane", "Beşiktaş", "Beykoz", "Şile"],
            "Silivri": ["Çatalca", "Büyükçekmece"],
            "Sultanbeyli": ["Pendik", "Kartal", "Sancaktepe", "Ümraniye"],
            "Sultangazi": ["Eyüpsultan", "Gaziosmanpaşa", "Kağıthane"],
            "Şile": ["Sarıyer"],
            "Şişli": ["Beyoğlu", "Beşiktaş", "Kağıthane", "Bayrampaşa", "Fatih"],
            "Tuzla": ["Pendik", "Kartal"],
            "Ümraniye": ["Küçükçekmece?", "Ataşehir", "Kadıköy", "Maltepe", "Sancaktepe", "Çekmeköy", "Üsküdar", "Beykoz"],
            "Üsküdar": ["Ümraniye", "Çekmeköy", "Beykoz", "Kadıköy", "Ataşehir"],
            "Zeytinburnu": ["Bakırköy", "Güngören", "Fatih", "Bayrampaşa"]
        },
        "maxRotationsPerPersonnel": 5
    }
    '''

In [11]:
df_station_locations

,Ekip Adı,lat,lon,Bölge,city,district,station_count,rural,double_station,double_station_with,Neighbourhood,District
0,ARN1,41.177852,28.745802,ARN,ARNAVUTKÖY,ARNAVUTKÖY MERKEZ,1,KENTSEL,False,NaN,Arnavutköy Merkez Mahallesi,Arnavutköy
1,ARN2,41.156037,28.623824,ARN,ARNAVUTKÖY,HADIMKÖY,1,KENTSEL,False,NaN,Hadımköy Mahallesi,Arnavutköy
2,ARN3,41.204411,28.716732,ARN,ARNAVUTKÖY,MAREŞAL FEVZİ ÇAKMAK,1,KENTSEL,False,NaN,Mareşal Fevzi Çakmak Mahallesi,Arnavutköy
3,ARN4,41.199450,28.760639,ARN,ARNAVUTKÖY,HİCRET,1,KENTSEL,False,NaN,Hicret Mahallesi,Arnavutköy
4,ARN6,41.185858,28.708606,ARN,ARNAVUTKÖY,TAŞOLUK,1,KENTSEL,False,NaN,Taşoluk Mahallesi,Arnavutköy
...,...,...,...,...,...,...,...,...,...,...,...,...
207,ZTB7,41.006756,28.897603,ZTB,ZEYTİNBURNU,SEYİTNİZAM,1,KENTSEL,False,NaN,Seyitnizam Mahallesi,Zeytinburnu
208,ARN5,41.265160,28.688026,ARN,ARNAVUTKÖY,TAYAKADIN,1,KIRSAL,False,NaN,Tayakadın Mahallesi,Arnavutköy
209,ÇTL4,41.413472,28.186711,ÇTL,ÇATALCA,FATİH,1,KIRSAL,False,NaN,Fatih Mahallesi,Çatalca
210,EYP6,41.253788,28.714775,EYP,ARNAVUTKÖY,TAYAKADIN,1,KIRSAL,True,EYP7,Tayakadın Mahallesi,Arnavutköy


In [12]:
def get_day_name_and_season(date):
    """
    Returns the current day name and season name.
    Day name is in uppercase (e.g., 'MONDAY').
    Season name is in uppercase (e.g., 'SUMMER').
    """
    date= pd.to_datetime(date, format='mixed')
    
    # Get day name
    day_name = date.strftime('%A') # e.g., 'WEDNESDAY'

    # Get season name
    def get_season(date):
        Y = date.year
        seasons = [
            ('Winter', dt.datetime(Y, 1, 1), dt.datetime(Y, 3, 21)),
            ('Spring', dt.datetime(Y, 3, 21), dt.datetime(Y, 6, 21)),
            ('Summer', dt.datetime(Y, 6, 21), dt.datetime(Y, 9, 23)),
            ('Autumn', dt.datetime(Y, 9, 23), dt.datetime(Y, 12, 21)),
            ('Winter', dt.datetime(Y, 12, 21), dt.datetime(Y+1, 1, 1))
        ]
        for season, start, end in seasons:
            if start <= date <= end:
                return season
        return 'UNKNOWN'

    season_name = get_season(date)

    return season_name, day_name


In [13]:
season, day= get_day_name_and_season('25-12-2025 14:45')

In [14]:
season

'Winter'

In [15]:
day

'Thursday'

In [16]:
json_input = '''
    {
        "stations": [
            {
                "id": "",
                "region": "",
                "importance": "",
                "assignedPersonnel": [
                    {"id": "", "name": "", "roles": [""], "assignedFrom": "", "homeStationId": "", "negativeStations": [], "preferredStations": [], "rotationCount": 0}
                ],
                "maxRotationCount": 5
            }
        ],
        "neighboringRegions" : {
            "Adalar": ["Kartal", "Pendik"],
            "Arnavutköy": ["Çatalca", "Esenler", "Başakşehir", "Eyüpsultan"],
            "Ataşehir": ["Üsküdar", "Kadıköy", "Maltepe", "Ümraniye"],
            "Avcılar": ["Küçükçekmece", "Bağcılar", "Başakşehir"],
            "Bağcılar": ["Küçükçekmece", "Bahçelievler", "Güngören", "Esenler", "Avcılar"],
            "Bahçelievler": ["Bağcılar", "Bakırköy", "Güngören"],
            "Bakırköy": ["Bahçelievler", "Güngören", "Küçükçekmece", "Esenyurt"],
            "Başakşehir": ["Arnavutköy", "Eyüpsultan", "Esenler", "Avcılar", "Küçükçekmece"],
            "Bayrampaşa": ["Gaziosmanpaşa", "Eyüpsultan", "Fatih", "Şişli"],
            "Beşiktaş": ["Şişli", "Beyoğlu", "Sarıyer", "Kağıthane"],
            "Beykoz": ["Ümraniye", "Çekmeköy", "Sancaktepe", "Üsküdar"],
            "Beylikdüzü": ["Büyükçekmece", "Esenyurt", "Avcılar"],
            "Beyoğlu": ["Şişli", "Kağıthane", "Fatih", "Beşiktaş", "Eyüpsultan"],
            "Büyükçekmece": ["Çatalca", "Silivri", "Esenyurt", "Beylikdüzü"],
            "Çatalca": ["Silivri", "Büyükçekmece", "Arnavutköy"],
            "Çekmeköy": ["Ümraniye", "Beykoz", "Üsküdar", "Sancaktepe"],
            "Esenler": ["Bağcılar", "Güngören", "Bakırköy", "Başakşehir", "Gaziosmanpaşa"],
            "Esenyurt": ["Küçükçekmece", "Avcılar", "Bakırköy", "Beylikdüzü", "Büyükçekmece"],
            "Eyüpsultan": ["Sarıyer", "Kağıthane", "Beyoğlu", "Gaziosmanpaşa", "Bayrampaşa", "Fatih", "Sultangazi", "Başakşehir", "Arnavutköy"],
            "Fatih": ["Beyoğlu", "Şişli", "Eminönü", "Esenler"],
            "Gaziosmanpaşa": ["Eyüpsultan", "Sultangazi", "Esenler", "Bayrampaşa"],
            "Güngören": ["Bağcılar", "Bahçelievler", "Bakırköy", "Esenler", "Zeytinburnu"],
            "Kadıköy": ["Ümraniye", "Ataşehir", "Maltepe", "Kartal", "Sancaktepe"],
            "Kağıthane": ["Şişli", "Beşiktaş", "Beyoğlu", "Eyüpsultan", "Sarıyer", "Sultangazi"],
            "Kartal": ["Maltepe", "Pendik", "Tuzla", "Adalar"],
            "Küçükçekmece": ["Avcılar", "Bağcılar", "Bakırköy", "Esenyurt", "Başakşehir"],
            "Maltepe": ["Kadıköy", "Kartal", "Pendik", "Sancaktepe", "Ümraniye"],
            "Pendik": ["Kartal", "Tuzla", "Sancaktepe", "Maltepe", "Ümraniye"],
            "Sancaktepe": ["Çekmeköy", "Ümraniye", "Kadıköy", "Maltepe", "Pendik", "Tuzla", "Kartal"],
            "Sarıyer": ["Eyüpsultan", "Kağıthane", "Beşiktaş", "Beykoz", "Şile"],
            "Silivri": ["Çatalca", "Büyükçekmece"],
            "Sultanbeyli": ["Pendik", "Kartal", "Sancaktepe", "Ümraniye"],
            "Sultangazi": ["Eyüpsultan", "Gaziosmanpaşa", "Kağıthane"],
            "Şile": ["Sarıyer"],
            "Şişli": ["Beyoğlu", "Beşiktaş", "Kağıthane", "Bayrampaşa", "Fatih"],
            "Tuzla": ["Pendik", "Kartal"],
            "Ümraniye": ["Üsküdar","Ataşehir", "Kadıköy", "Maltepe", "Sancaktepe", "Çekmeköy",  "Beykoz"],
            "Üsküdar": ["Ümraniye", "Çekmeköy", "Beykoz", "Kadıköy", "Ataşehir"],
            "Zeytinburnu": ["Bakırköy", "Güngören", "Fatih", "Bayrampaşa"]
        },
        "maxRotationsPerPersonnel": 5
    }
'''

In [17]:
data = json.loads(json_input)

In [18]:
data.keys()

dict_keys(['stations', 'neighboringRegions', 'maxRotationsPerPersonnel'])

In [19]:
pd.DataFrame(data['stations'])

,id,region,importance,assignedPersonnel,maxRotationCount
0,,,,"[{'id': '', 'name': '', 'roles': [''], 'assign...",5


In [20]:
data["stations"][0].keys()

dict_keys(['id', 'region', 'importance', 'assignedPersonnel', 'maxRotationCount'])

In [21]:

data["stations"][0]["assignedPersonnel"]

[{'id': '',
  'name': '',
  'roles': [''],
  'assignedFrom': '',
  'homeStationId': '',
  'negativeStations': [],
  'preferredStations': [],
  'rotationCount': 0}]

In [22]:
df_shifts= pd.read_excel(rf"C:\Users\mkaya\Downloads\Personel-Nöbet-Listesi (31).xls", header=None)

In [23]:
df_signs= pd.read_excel(r"C:\Users\mkaya\Downloads\Personel-Imza-Defteri (8).xls", header=None)

In [24]:
df_signs= df_signs[df_signs[2].notna()]

In [25]:
df_signs

,0,1,2,3,4,5,6,7,8,9,10,11
6,NaN,Sıra No,Adı Soyadı,NaN,NaN,Unvan,Başlama Tarihi,Giriş Açıklama,Bitiş Tarihi,NaN,Çıkış Açıklama,Ekip
7,NaN,1,ABDULALİM KOÇER,NaN,NaN,Ekip Sorumlusu,NaN,NaN,NaN,NaN,NaN,BEŞ5
8,NaN,2,ABDULKADİR AKSÖZ,NaN,NaN,Sürücü,NaN,NaN,NaN,NaN,NaN,OTHYTKN2
9,NaN,3,ABDULKADİR GÜNDÜZ,NaN,NaN,Diğer,NaN,NaN,NaN,NaN,NaN,AVRUPABAŞHEKİMLİK
10,NaN,4,ABDULKADİR KEKEÇ,NaN,NaN,Yardımcı Sağlık Personeli,25/06/2025 08:07:20,NaN,NaN,NaN,NaN,KĞT5
...,...,...,...,...,...,...,...,...,...,...,...,...
2055,NaN,2049,ZÜBEYDE ÖZELSAĞIROĞLU,NaN,NaN,Diğer,NaN,NaN,NaN,NaN,NaN,ASOSARAÇ1
2056,NaN,2050,ZÜLBİYE SARI,NaN,NaN,Ekip Sorumlusu,25/06/2025 07:50:42,NaN,NaN,NaN,NaN,BÇK2
2057,NaN,2051,ZÜLEYHA KOÇ,NaN,NaN,Diğer,NaN,NaN,NaN,NaN,NaN,AVRUPAİŞÇİSAĞLIĞI
2058,NaN,2052,ZÜLEYHA ŞENTÜRK,NaN,NaN,Diğer,NaN,NaN,NaN,NaN,NaN,AVUPAKALİTE


In [26]:
df_signs= df_signs.dropna(how='all',axis=1)

In [27]:
df_signs.columns= df_signs.iloc[0]

In [28]:
df_signs= df_signs[1:]

In [29]:
df_shifts

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,NaN,NaN,NaN,NaN,T.C.\nSağlık Bakanlığı\nPersonel Nöbet Listesi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06.2025,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2400,NaN,AYŞE,KARTAL,NaN,NaN,26669574218,24 Saat,Ekip Sorumlusu,İLK VE ACİL YARDIM,657/4A Memur,05300530112,25-06-2025 08:00,26-06-2025 08:00,NaN,NaN,NaN
2401,NaN,BATUHAN,ATİLLA,NaN,NaN,11360158140,12 Saat,Sürücü,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05321668024,25-06-2025 20:00,26-06-2025 08:00,NaN,NaN,NaN
2402,NaN,GÖKHAN,ÜNVER,NaN,NaN,11713462870,24 Saat,Ekip Sorumlusu,İLK VE ACİL YARDIM,657/4A Memur,05447985514,25-06-2025 08:00,26-06-2025 08:00,NaN,NaN,NaN
2403,NaN,MEHMET,YEDİKARDEŞ,NaN,NaN,58687462512,12 Saat,Sürücü,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05372550657,25-06-2025 08:00,25-06-2025 20:00,NaN,NaN,NaN


In [30]:
df_shifts

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,NaN,NaN,NaN,NaN,T.C.\nSağlık Bakanlığı\nPersonel Nöbet Listesi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06.2025,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2400,NaN,AYŞE,KARTAL,NaN,NaN,26669574218,24 Saat,Ekip Sorumlusu,İLK VE ACİL YARDIM,657/4A Memur,05300530112,25-06-2025 08:00,26-06-2025 08:00,NaN,NaN,NaN
2401,NaN,BATUHAN,ATİLLA,NaN,NaN,11360158140,12 Saat,Sürücü,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05321668024,25-06-2025 20:00,26-06-2025 08:00,NaN,NaN,NaN
2402,NaN,GÖKHAN,ÜNVER,NaN,NaN,11713462870,24 Saat,Ekip Sorumlusu,İLK VE ACİL YARDIM,657/4A Memur,05447985514,25-06-2025 08:00,26-06-2025 08:00,NaN,NaN,NaN
2403,NaN,MEHMET,YEDİKARDEŞ,NaN,NaN,58687462512,12 Saat,Sürücü,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05372550657,25-06-2025 08:00,25-06-2025 20:00,NaN,NaN,NaN


In [31]:
df_signs= df_signs[df_signs['Başlama Tarihi'].notna()]

In [32]:
df_signs

6,Sıra No,Adı Soyadı,Unvan,Başlama Tarihi,Giriş Açıklama,Bitiş Tarihi,Çıkış Açıklama,Ekip
10,4,ABDULKADİR KEKEÇ,Yardımcı Sağlık Personeli,25/06/2025 08:07:20,NaN,NaN,NaN,KĞT5
17,11,ABDULLAH EMRE ŞENGÜN,Sürücü,25/06/2025 07:58:01,NaN,NaN,NaN,BEY1
23,17,ABDULSAMET AKTEPE,Sürücü,25/06/2025 08:04:44,NaN,NaN,NaN,KÇK14
32,26,ADEM MUSULLU,Sürücü,25/06/2025 08:07:32,NaN,NaN,NaN,BDZ4
33,27,ADEM PALA,Yardımcı Sağlık Personeli,25/06/2025 08:01:20,.,NaN,NaN,GOP4
...,...,...,...,...,...,...,...,...
2037,2031,ZEYNEP ÇINAR,Diğer,25/06/2025 07:53:31,NaN,NaN,NaN,AVC6
2048,2042,ZEYNEP YILMAZ,Yardımcı Sağlık Personeli,25/06/2025 08:23:36,,NaN,NaN,ESY12
2054,2048,ZUHURİ AĞA,Yardımcı Sağlık Personeli,25/06/2025 07:55:42,NaN,NaN,NaN,.KRİZ GÜNEY
2056,2050,ZÜLBİYE SARI,Ekip Sorumlusu,25/06/2025 07:50:42,NaN,NaN,NaN,BÇK2


In [33]:
df_station_locations

,Ekip Adı,lat,lon,Bölge,city,district,station_count,rural,double_station,double_station_with,Neighbourhood,District
0,ARN1,41.177852,28.745802,ARN,ARNAVUTKÖY,ARNAVUTKÖY MERKEZ,1,KENTSEL,False,NaN,Arnavutköy Merkez Mahallesi,Arnavutköy
1,ARN2,41.156037,28.623824,ARN,ARNAVUTKÖY,HADIMKÖY,1,KENTSEL,False,NaN,Hadımköy Mahallesi,Arnavutköy
2,ARN3,41.204411,28.716732,ARN,ARNAVUTKÖY,MAREŞAL FEVZİ ÇAKMAK,1,KENTSEL,False,NaN,Mareşal Fevzi Çakmak Mahallesi,Arnavutköy
3,ARN4,41.199450,28.760639,ARN,ARNAVUTKÖY,HİCRET,1,KENTSEL,False,NaN,Hicret Mahallesi,Arnavutköy
4,ARN6,41.185858,28.708606,ARN,ARNAVUTKÖY,TAŞOLUK,1,KENTSEL,False,NaN,Taşoluk Mahallesi,Arnavutköy
...,...,...,...,...,...,...,...,...,...,...,...,...
207,ZTB7,41.006756,28.897603,ZTB,ZEYTİNBURNU,SEYİTNİZAM,1,KENTSEL,False,NaN,Seyitnizam Mahallesi,Zeytinburnu
208,ARN5,41.265160,28.688026,ARN,ARNAVUTKÖY,TAYAKADIN,1,KIRSAL,False,NaN,Tayakadın Mahallesi,Arnavutköy
209,ÇTL4,41.413472,28.186711,ÇTL,ÇATALCA,FATİH,1,KIRSAL,False,NaN,Fatih Mahallesi,Çatalca
210,EYP6,41.253788,28.714775,EYP,ARNAVUTKÖY,TAYAKADIN,1,KIRSAL,True,EYP7,Tayakadın Mahallesi,Arnavutköy


In [34]:
# %load staff_shift_file_celaning.py
def staff_shift_file_cleaning(df):

    # WARNING: READ YOUR EXCEL FILE WITH header=None , F.e = pd.read_excel("df.xlsx", header=None)

    # Identify station rows: col1 has value, cols 2–4 are empty
    station_mask = df[1].notna() & df[[2, 3, 4]].isna().all(axis=1)
    df["RawStation"] = df[1].where(station_mask)

    # Forward fill station names
    df["İstasyon"] = df["RawStation"].ffill()
    df.columns= df.loc[df[df[1] == 'İsim'].index[0]]
    df= df[[col for col in df.columns if pd.notna(col)]]
    df.rename(columns={df.columns[-1]: 'İstasyon'}, inplace=True)
    df= df[(df['Kimlik No'].notna()) & (df['Kimlik No'] != 'Kimlik No')]
    
    return df

In [35]:
df_shifts= staff_shift_file_cleaning(df_shifts)

C:\Users\mkaya\AppData\Local\Temp\ipykernel_2992\2585991137.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={df.columns[-1]: 'İstasyon'}, inplace=True)


In [36]:
df_shifts= df_shifts[(df_shifts['Başlangıç Tarihi'].notna()) & ((pd.to_datetime(df_shifts['Bitiş Tarihi'], format= 'mixed') - pd.to_datetime(df_shifts['Başlangıç Tarihi'], format='mixed')) > pd.Timedelta(hours=8))]

In [37]:
df_shifts

7,İsim,Soyisim,Kimlik No,Mesai,Görev,Branş,Kadro,Telefon,Başlangıç Tarihi,Bitiş Tarihi,Açıklama,İstasyon
8,FIRAT,AYAN,17680618838,16 Saat,Diğer,YOK,657/4A Memur,05318476097,25-06-2025 16:00,26-06-2025 08:00,NaN,.ASKOM ARNAVUTKÖY
14,EROL,SÜLKÜ,53821394308,24 Saat,Diğer,TOPLUM SAĞLIĞI,657/4A Memur,05433667564,25-06-2025 08:00,26-06-2025 08:00,NaN,.ASKOM AVCILAR
21,NUREDDİN,ÇELİK,13903505602,16 Saat,Yardımcı Sağlık Personeli,İLK VE ACİL YARDIM,657/4A Memur,05066325741,25-06-2025 16:00,26-06-2025 08:00,NaN,.ASKOM BAĞCILAR
27,ÖZGÜR,BOZDAĞ,19754413790,16 Saat,Diğer,YOK,657/4A Memur,05069258784,25-06-2025 16:00,26-06-2025 08:00,NaN,.ASKOM BAHÇELİEVLER
33,HACER,TUNCER BOZ,12152857326,12 Saat,Yardımcı Sağlık Personeli,İLK VE ACİL YARDIM,657/4A Memur,05349640306,25-06-2025 08:00,25-06-2025 20:00,NaN,.ASKOM BAKIRKÖY*
...,...,...,...,...,...,...,...,...,...,...,...,...
2400,AYŞE,KARTAL,26669574218,24 Saat,Ekip Sorumlusu,İLK VE ACİL YARDIM,657/4A Memur,05300530112,25-06-2025 08:00,26-06-2025 08:00,NaN,Z.BURNU 7
2401,BATUHAN,ATİLLA,11360158140,12 Saat,Sürücü,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05321668024,25-06-2025 20:00,26-06-2025 08:00,NaN,Z.BURNU 7
2402,GÖKHAN,ÜNVER,11713462870,24 Saat,Ekip Sorumlusu,İLK VE ACİL YARDIM,657/4A Memur,05447985514,25-06-2025 08:00,26-06-2025 08:00,NaN,Z.BURNU 7
2403,MEHMET,YEDİKARDEŞ,58687462512,12 Saat,Sürücü,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05372550657,25-06-2025 08:00,25-06-2025 20:00,NaN,Z.BURNU 7


In [38]:
df_shifts[df_shifts['İstasyon'].astype(str).str.strip().isin(df_station_locations['Ekip Adı'].astype(str).str.strip())]

7,İsim,Soyisim,Kimlik No,Mesai,Görev,Branş,Kadro,Telefon,Başlangıç Tarihi,Bitiş Tarihi,Açıklama,İstasyon


In [39]:
df_shifts[pd.to_datetime(df_shifts['Başlangıç Tarihi'], format= 'mixed').dt.hour <= 12]

7,İsim,Soyisim,Kimlik No,Mesai,Görev,Branş,Kadro,Telefon,Başlangıç Tarihi,Bitiş Tarihi,Açıklama,İstasyon
14,EROL,SÜLKÜ,53821394308,24 Saat,Diğer,TOPLUM SAĞLIĞI,657/4A Memur,05433667564,25-06-2025 08:00,26-06-2025 08:00,NaN,.ASKOM AVCILAR
33,HACER,TUNCER BOZ,12152857326,12 Saat,Yardımcı Sağlık Personeli,İLK VE ACİL YARDIM,657/4A Memur,05349640306,25-06-2025 08:00,25-06-2025 20:00,NaN,.ASKOM BAKIRKÖY*
39,BERNA,AYDIN,35473691738,24 Saat,Yardımcı Sağlık Personeli,İLK VE ACİL YARDIM,657/4A Memur,05336441561,25-06-2025 08:00,26-06-2025 08:00,NaN,.ASKOM BAŞAKŞEHİR ÇAM SAKURA HAST.
40,NURAY,AKGÜL,28600437538,24 Saat,Yardımcı Sağlık Personeli,İLK VE ACİL YARDIM,657/4A Memur,05457889902,25-06-2025 08:00,26-06-2025 08:00,NaN,.ASKOM BAŞAKŞEHİR ÇAM SAKURA HAST.
52,ALPEREN,YÜCEL,35273084990,24 Saat,Diğer,YOK,657/4A Memur,05419615793,25-06-2025 08:00,26-06-2025 08:00,NaN,.ASKOM BÜYÜKÇEKMECE
...,...,...,...,...,...,...,...,...,...,...,...,...
2393,TUĞÇE,ŞAHİN,17362615056,24 Saat,Ekip Sorumlusu,İLK VE ACİL YARDIM,657/4A Memur,05445479115,25-06-2025 08:00,26-06-2025 08:00,NaN,Z.BURNU 6
2394,YUNUS,KILIÇ,64321260598,24 Saat,Sürücü,İLK VE ACİL YARDIM,657/4A Memur,05436687855,25-06-2025 08:00,26-06-2025 08:00,NaN,Z.BURNU 6
2400,AYŞE,KARTAL,26669574218,24 Saat,Ekip Sorumlusu,İLK VE ACİL YARDIM,657/4A Memur,05300530112,25-06-2025 08:00,26-06-2025 08:00,NaN,Z.BURNU 7
2402,GÖKHAN,ÜNVER,11713462870,24 Saat,Ekip Sorumlusu,İLK VE ACİL YARDIM,657/4A Memur,05447985514,25-06-2025 08:00,26-06-2025 08:00,NaN,Z.BURNU 7


In [40]:
df_shifts['İsim Soyisim']= df_shifts['İsim'].astype(str).str.strip() + ' ' + df_shifts['Soyisim'].astype(str).str.strip()

In [41]:
df_signs['Adı Soyadı']= df_signs['Adı Soyadı'].astype(str).str.strip()

In [42]:
df_signs

6,Sıra No,Adı Soyadı,Unvan,Başlama Tarihi,Giriş Açıklama,Bitiş Tarihi,Çıkış Açıklama,Ekip
10,4,ABDULKADİR KEKEÇ,Yardımcı Sağlık Personeli,25/06/2025 08:07:20,NaN,NaN,NaN,KĞT5
17,11,ABDULLAH EMRE ŞENGÜN,Sürücü,25/06/2025 07:58:01,NaN,NaN,NaN,BEY1
23,17,ABDULSAMET AKTEPE,Sürücü,25/06/2025 08:04:44,NaN,NaN,NaN,KÇK14
32,26,ADEM MUSULLU,Sürücü,25/06/2025 08:07:32,NaN,NaN,NaN,BDZ4
33,27,ADEM PALA,Yardımcı Sağlık Personeli,25/06/2025 08:01:20,.,NaN,NaN,GOP4
...,...,...,...,...,...,...,...,...
2037,2031,ZEYNEP ÇINAR,Diğer,25/06/2025 07:53:31,NaN,NaN,NaN,AVC6
2048,2042,ZEYNEP YILMAZ,Yardımcı Sağlık Personeli,25/06/2025 08:23:36,,NaN,NaN,ESY12
2054,2048,ZUHURİ AĞA,Yardımcı Sağlık Personeli,25/06/2025 07:55:42,NaN,NaN,NaN,.KRİZ GÜNEY
2056,2050,ZÜLBİYE SARI,Ekip Sorumlusu,25/06/2025 07:50:42,NaN,NaN,NaN,BÇK2


In [43]:
df_shifts

7,İsim,Soyisim,Kimlik No,Mesai,Görev,Branş,Kadro,Telefon,Başlangıç Tarihi,Bitiş Tarihi,Açıklama,İstasyon,İsim Soyisim
8,FIRAT,AYAN,17680618838,16 Saat,Diğer,YOK,657/4A Memur,05318476097,25-06-2025 16:00,26-06-2025 08:00,NaN,.ASKOM ARNAVUTKÖY,FIRAT AYAN
14,EROL,SÜLKÜ,53821394308,24 Saat,Diğer,TOPLUM SAĞLIĞI,657/4A Memur,05433667564,25-06-2025 08:00,26-06-2025 08:00,NaN,.ASKOM AVCILAR,EROL SÜLKÜ
21,NUREDDİN,ÇELİK,13903505602,16 Saat,Yardımcı Sağlık Personeli,İLK VE ACİL YARDIM,657/4A Memur,05066325741,25-06-2025 16:00,26-06-2025 08:00,NaN,.ASKOM BAĞCILAR,NUREDDİN ÇELİK
27,ÖZGÜR,BOZDAĞ,19754413790,16 Saat,Diğer,YOK,657/4A Memur,05069258784,25-06-2025 16:00,26-06-2025 08:00,NaN,.ASKOM BAHÇELİEVLER,ÖZGÜR BOZDAĞ
33,HACER,TUNCER BOZ,12152857326,12 Saat,Yardımcı Sağlık Personeli,İLK VE ACİL YARDIM,657/4A Memur,05349640306,25-06-2025 08:00,25-06-2025 20:00,NaN,.ASKOM BAKIRKÖY*,HACER TUNCER BOZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2400,AYŞE,KARTAL,26669574218,24 Saat,Ekip Sorumlusu,İLK VE ACİL YARDIM,657/4A Memur,05300530112,25-06-2025 08:00,26-06-2025 08:00,NaN,Z.BURNU 7,AYŞE KARTAL
2401,BATUHAN,ATİLLA,11360158140,12 Saat,Sürücü,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05321668024,25-06-2025 20:00,26-06-2025 08:00,NaN,Z.BURNU 7,BATUHAN ATİLLA
2402,GÖKHAN,ÜNVER,11713462870,24 Saat,Ekip Sorumlusu,İLK VE ACİL YARDIM,657/4A Memur,05447985514,25-06-2025 08:00,26-06-2025 08:00,NaN,Z.BURNU 7,GÖKHAN ÜNVER
2403,MEHMET,YEDİKARDEŞ,58687462512,12 Saat,Sürücü,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05372550657,25-06-2025 08:00,25-06-2025 20:00,NaN,Z.BURNU 7,MEHMET YEDİKARDEŞ


In [44]:
df_shifts= df_shifts[df_shifts['İsim Soyisim'].isin(df_signs['Adı Soyadı'].to_list())]

In [45]:
df_shifts.loc[(df_shifts['Görev']=='Ekip Sorumlusu') | (df_shifts['Görev']=='Yardımcı Sağlık Personeli'), 'Görev'] = 'medic'
df_shifts.loc[(df_shifts['Görev']=='Sürücü') , 'Görev'] = 'driver'

In [46]:
import pandas as pd

In [47]:
df_team_code_dict= pd.read_csv(r"C:\Users\mkaya\OneDrive\Masaüstü\istanbul112_hidden\data\keywords\team_code_match\eu_team_code.csv")
df_team_code_dict['112ONLINE']= df_team_code_dict['112ONLINE'].str.strip()
df_team_code_dict['ASOSYON']= df_team_code_dict['ASOS'].str.strip()

In [48]:
df_team_code_dict= dict(zip(df_team_code_dict['112ONLINE'], df_team_code_dict['ASOS']))

In [49]:
df_team_code_dict

{'ARNKÖY 1': 'ARN1',
 'ARNKÖY 2': 'ARN2',
 'ARNKÖY 3': 'ARN3',
 'ARNKÖY 4': 'ARN4',
 'ARNKÖY 5': 'ARN5',
 'ARNKÖY 6': 'ARN6',
 'ARNKÖY 7': 'ARN7',
 'AVCILAR 1': 'AVC1',
 'AVCILAR 2': 'AVC2',
 'AVCILAR 3': 'AVC3',
 'AVCILAR 4': 'AVC4',
 'AVCILAR 5': 'AVC5',
 'AVCILAR 6': 'AVC6',
 'AVCILAR 7': 'AVC7',
 'AVCILAR 8': 'AVC8',
 'AVCILAR 9': 'AVC9',
 'B.EVLER 01': 'BHC1',
 'B.EVLER 02': 'BHC2',
 'B.EVLER 03': 'BHC3',
 'B.EVLER 04': 'BHC4',
 'B.EVLER 05': 'BHC5',
 'B.EVLER 06': 'BHC6',
 'B.EVLER 07': 'BHC7',
 'B.EVLER 08': 'BHC8',
 'B.EVLER 09': 'BHC9',
 'B.EVLER 10': 'BHC10',
 'B.EVLER 11': 'BHC11',
 'BAKIRKÖY 1': 'BKR1',
 'BAKIRKÖY 2': 'BKR2',
 'BAKIRKÖY 3': 'BKR3',
 'BAKIRKÖY 4': 'BKR4',
 'BAKIRKÖY 5': 'BKR5',
 'BAKIRKÖY 6': 'BKR6',
 'BAKIRKÖY 7': 'BKR7',
 'BAKIRKÖY 8': 'BKR8',
 'BAYRAMP 1': 'BYR1',
 'BAYRAMP 2': 'BYR2',
 'BAYRAMP 3': 'BYR3',
 'BAYRAMP 4': 'BYR4',
 'BAYRAMP 5': 'BYR5',
 'BAYRAMP 6': 'BYR6',
 'BAYRAMP 7': 'BYR7',
 'BAĞCILAR 01': 'BAG1',
 'BAĞCILAR 10': 'BAG10',
 'BAĞCILAR 11

In [ ]:
'ARNKÖY':'ARNAVUTKÖY',
'AVCILAR':'AVCILAR',
'B.EVLER':'BAHÇELİEVLER',
'BAKIRKÖY':'BAKIRKÖY',


In [50]:
df_shifts['İstasyon']= df_shifts['İstasyon'].str.strip().map(df_team_code_dict)

C:\Users\mkaya\AppData\Local\Temp\ipykernel_2992\3769806514.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shifts['İstasyon']= df_shifts['İstasyon'].str.strip().map(df_team_code_dict)


In [51]:
df_shifts= df_shifts[df_shifts['İstasyon'].notna()]

In [52]:
df_shifts

7,İsim,Soyisim,Kimlik No,Mesai,Görev,Branş,Kadro,Telefon,Başlangıç Tarihi,Bitiş Tarihi,Açıklama,İstasyon,İsim Soyisim
396,AHMET,AKAR,54001069184,12 Saat,driver,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05468122215,25-06-2025 08:00,25-06-2025 20:00,NaN,ARN1,AHMET AKAR
402,FEVZİ,BİÇER,22702724848,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05353780732,25-06-2025 08:00,26-06-2025 08:00,NaN,ARN2,FEVZİ BİÇER
403,HOZAN,EREN,25303629178,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05458503007,25-06-2025 08:00,26-06-2025 08:00,NaN,ARN2,HOZAN EREN
404,SAMET,SÜME,30670657914,12 Saat,driver,8322.04 AMBULANS SÜRÜCÜSÜ,Sürekli İşçi (696 sayılı KHK),05422820029,25-06-2025 08:00,25-06-2025 20:00,NaN,ARN2,SAMET SÜME
411,AYŞEANA,KÖSEOĞLU,69445165774,24 Saat,medic,İLK VE ACİL YARDIM,657/4B Sözleşmeli Personel (663 sayılı KHK 45/A),05451052205,25-06-2025 08:00,26-06-2025 08:00,NaN,ARN3,AYŞEANA KÖSEOĞLU
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,NESİBE ŞEYDA,ÇIRAK,39352076050,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05352358517,25-06-2025 08:00,26-06-2025 08:00,NaN,ZTB5,NESİBE ŞEYDA ÇIRAK
2393,TUĞÇE,ŞAHİN,17362615056,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05445479115,25-06-2025 08:00,26-06-2025 08:00,NaN,ZTB6,TUĞÇE ŞAHİN
2394,YUNUS,KILIÇ,64321260598,24 Saat,driver,İLK VE ACİL YARDIM,657/4A Memur,05436687855,25-06-2025 08:00,26-06-2025 08:00,NaN,ZTB6,YUNUS KILIÇ
2400,AYŞE,KARTAL,26669574218,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05300530112,25-06-2025 08:00,26-06-2025 08:00,NaN,ZTB7,AYŞE KARTAL


In [53]:
df_station_locations['Ekip Adı']= df_station_locations['Ekip Adı'].str.strip()

In [54]:
df_station_locations

,Ekip Adı,lat,lon,Bölge,city,district,station_count,rural,double_station,double_station_with,Neighbourhood,District
0,ARN1,41.177852,28.745802,ARN,ARNAVUTKÖY,ARNAVUTKÖY MERKEZ,1,KENTSEL,False,NaN,Arnavutköy Merkez Mahallesi,Arnavutköy
1,ARN2,41.156037,28.623824,ARN,ARNAVUTKÖY,HADIMKÖY,1,KENTSEL,False,NaN,Hadımköy Mahallesi,Arnavutköy
2,ARN3,41.204411,28.716732,ARN,ARNAVUTKÖY,MAREŞAL FEVZİ ÇAKMAK,1,KENTSEL,False,NaN,Mareşal Fevzi Çakmak Mahallesi,Arnavutköy
3,ARN4,41.199450,28.760639,ARN,ARNAVUTKÖY,HİCRET,1,KENTSEL,False,NaN,Hicret Mahallesi,Arnavutköy
4,ARN6,41.185858,28.708606,ARN,ARNAVUTKÖY,TAŞOLUK,1,KENTSEL,False,NaN,Taşoluk Mahallesi,Arnavutköy
...,...,...,...,...,...,...,...,...,...,...,...,...
207,ZTB7,41.006756,28.897603,ZTB,ZEYTİNBURNU,SEYİTNİZAM,1,KENTSEL,False,NaN,Seyitnizam Mahallesi,Zeytinburnu
208,ARN5,41.265160,28.688026,ARN,ARNAVUTKÖY,TAYAKADIN,1,KIRSAL,False,NaN,Tayakadın Mahallesi,Arnavutköy
209,ÇTL4,41.413472,28.186711,ÇTL,ÇATALCA,FATİH,1,KIRSAL,False,NaN,Fatih Mahallesi,Çatalca
210,EYP6,41.253788,28.714775,EYP,ARNAVUTKÖY,TAYAKADIN,1,KIRSAL,True,EYP7,Tayakadın Mahallesi,Arnavutköy


In [55]:
region_dict= dict(zip(df_station_locations['Ekip Adı'], df_station_locations['District']))

In [86]:
region_dict

{'ARN1': 'Arnavutköy',
 'ARN2': 'Arnavutköy',
 'ARN3': 'Arnavutköy',
 'ARN4': 'Arnavutköy',
 'ARN6': 'Arnavutköy',
 'ARN7': 'Arnavutköy',
 'AVC1': 'Avcılar',
 'AVC2': 'Avcılar',
 'AVC3': 'Avcılar',
 'AVC4': 'Avcılar',
 'AVC5': 'Avcılar',
 'AVC6': 'Esenyurt',
 'AVC7': 'Avcılar',
 'AVC8': 'Avcılar',
 'AVC9': 'Beylikdüzü',
 'BAG1': 'Bağcılar',
 'BAG10': 'Bağcılar',
 'BAG11': 'Bağcılar',
 'BAG12': 'Bağcılar',
 'BAG2': 'Bağcılar',
 'BAG3': 'Bağcılar',
 'BAG4': 'Bağcılar',
 'BAG5': 'Bağcılar',
 'BAG6': 'Bağcılar',
 'BAG7': 'Bağcılar',
 'BAG8': 'Bağcılar',
 'BAG9': 'Bağcılar',
 'BEŞ1': 'Beşiktaş',
 'BEŞ2': 'Beşiktaş',
 'BEŞ3': 'Beşiktaş',
 'BEŞ4': 'Beşiktaş',
 'BEŞ5': 'Beşiktaş',
 'BEŞ6': 'Beşiktaş',
 'BEY1': 'Beyoğlu',
 'BEY2': 'Şişli',
 'BEY3': 'Beyoğlu',
 'BEY4': 'Beyoğlu',
 'BEY5': 'Beyoğlu',
 'BEY6': 'Beyoğlu',
 'BÇK1': 'Büyükçekmece',
 'BÇK2': 'Büyükçekmece',
 'BÇK3': 'Büyükçekmece',
 'BÇK4': 'Büyükçekmece',
 'BÇK5': 'Büyükçekmece',
 'BÇK6': 'Büyükçekmece',
 'BÇK7': 'Büyükçekmece',
 'BD

In [56]:
df_shifts['region']= df_shifts['İstasyon'].map(region_dict)

In [57]:
df_shifts= df_shifts[pd.to_datetime(df_shifts['Başlangıç Tarihi']).dt.hour < 11]

C:\Users\mkaya\AppData\Local\Temp\ipykernel_2992\885263242.py:1: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_shifts= df_shifts[pd.to_datetime(df_shifts['Başlangıç Tarihi']).dt.hour < 11]


In [58]:
df_shifts

7,İsim,Soyisim,Kimlik No,Mesai,Görev,Branş,Kadro,Telefon,Başlangıç Tarihi,Bitiş Tarihi,Açıklama,İstasyon,İsim Soyisim,region
396,AHMET,AKAR,54001069184,12 Saat,driver,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05468122215,25-06-2025 08:00,25-06-2025 20:00,NaN,ARN1,AHMET AKAR,Arnavutköy
402,FEVZİ,BİÇER,22702724848,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05353780732,25-06-2025 08:00,26-06-2025 08:00,NaN,ARN2,FEVZİ BİÇER,Arnavutköy
403,HOZAN,EREN,25303629178,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05458503007,25-06-2025 08:00,26-06-2025 08:00,NaN,ARN2,HOZAN EREN,Arnavutköy
404,SAMET,SÜME,30670657914,12 Saat,driver,8322.04 AMBULANS SÜRÜCÜSÜ,Sürekli İşçi (696 sayılı KHK),05422820029,25-06-2025 08:00,25-06-2025 20:00,NaN,ARN2,SAMET SÜME,Arnavutköy
411,AYŞEANA,KÖSEOĞLU,69445165774,24 Saat,medic,İLK VE ACİL YARDIM,657/4B Sözleşmeli Personel (663 sayılı KHK 45/A),05451052205,25-06-2025 08:00,26-06-2025 08:00,NaN,ARN3,AYŞEANA KÖSEOĞLU,Arnavutköy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,NESİBE ŞEYDA,ÇIRAK,39352076050,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05352358517,25-06-2025 08:00,26-06-2025 08:00,NaN,ZTB5,NESİBE ŞEYDA ÇIRAK,Zeytinburnu
2393,TUĞÇE,ŞAHİN,17362615056,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05445479115,25-06-2025 08:00,26-06-2025 08:00,NaN,ZTB6,TUĞÇE ŞAHİN,Zeytinburnu
2394,YUNUS,KILIÇ,64321260598,24 Saat,driver,İLK VE ACİL YARDIM,657/4A Memur,05436687855,25-06-2025 08:00,26-06-2025 08:00,NaN,ZTB6,YUNUS KILIÇ,Zeytinburnu
2400,AYŞE,KARTAL,26669574218,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05300530112,25-06-2025 08:00,26-06-2025 08:00,NaN,ZTB7,AYŞE KARTAL,Zeytinburnu


In [59]:
data

{'stations': [{'id': '',
   'region': '',
   'importance': '',
   'assignedPersonnel': [{'id': '',
     'name': '',
     'roles': [''],
     'assignedFrom': '',
     'homeStationId': '',
     'negativeStations': [],
     'preferredStations': [],
     'rotationCount': 0}],
   'maxRotationCount': 5}],
 'neighboringRegions': {'Adalar': ['Kartal', 'Pendik'],
  'Arnavutköy': ['Çatalca', 'Esenler', 'Başakşehir', 'Eyüpsultan'],
  'Ataşehir': ['Üsküdar', 'Kadıköy', 'Maltepe', 'Ümraniye'],
  'Avcılar': ['Küçükçekmece', 'Bağcılar', 'Başakşehir'],
  'Bağcılar': ['Küçükçekmece',
   'Bahçelievler',
   'Güngören',
   'Esenler',
   'Avcılar'],
  'Bahçelievler': ['Bağcılar', 'Bakırköy', 'Güngören'],
  'Bakırköy': ['Bahçelievler', 'Güngören', 'Küçükçekmece', 'Esenyurt'],
  'Başakşehir': ['Arnavutköy',
   'Eyüpsultan',
   'Esenler',
   'Avcılar',
   'Küçükçekmece'],
  'Bayrampaşa': ['Gaziosmanpaşa', 'Eyüpsultan', 'Fatih', 'Şişli'],
  'Beşiktaş': ['Şişli', 'Beyoğlu', 'Sarıyer', 'Kağıthane'],
  'Beykoz': 

In [60]:
season, day= get_day_name_and_season(df_shifts.iloc[0]["Başlangıç Tarihi"])

In [61]:
season, day

('Summer', 'Wednesday')

In [62]:
df_shifts['Başlangıç Tarihi'].iloc[0]

'25-06-2025 08:00'

In [63]:
season, day

('Summer', 'Wednesday')

In [64]:
scoring_file_name= f"{season}_{day}_total scores.xlsx"

In [65]:
df_scorings= pd.read_excel(rf"C:\Users\mkaya\OneDrive\Masaüstü\istanbul112_hidden\data\case_reports\europe\parquet_files\team_case_intensities\overall\{scoring_file_name}")

In [66]:
df_scorings

,Unnamed: 0,index,Ekip No,total_distance,field_operation_time,hospital_delivery_time,case_response_time,yas_score,cinsiyet_score,yeni_dogan_score,...,case_count_logarithmic_score_MinMax_score,case_count_logarithmic_score_MinMax_score_z_score,teams_population_density_logarithmic_score,teams_population_density_logarithmic_score_MinMax_score,teams_population_density_logarithmic_score_MinMax_score_z_score,total_score,strategic_score,total_score_z,strategic_score_z,station_expendable
0,0,102,ESY5,15.321716,1705.467972,841.280177,891.435094,39.681936,73.595506,25.172861,...,1.000000,3.336389,4.849307,0.912457,1.568817,25105.854797,1.926214,4.921827,-1.181821,Non-Expendable
1,1,101,ESY3,16.187135,1804.872568,722.599260,1014.164519,40.408671,74.646560,25.047125,...,0.978497,2.900107,4.976190,1.000000,2.887967,23669.687726,2.120259,4.209444,-0.714030,Non-Expendable
2,2,99,ESY14,12.193798,1758.070866,777.571702,868.344288,39.532523,75.531915,25.000000,...,0.900457,1.316690,4.720795,0.823790,0.232733,23343.681759,2.404106,4.047735,-0.029750,Non-Expendable
3,3,94,ESY1,16.519174,1657.154448,757.614035,871.449505,37.307183,74.007561,25.000000,...,0.973539,2.799495,4.808144,0.884057,1.140867,22348.994302,1.919536,3.554339,-1.197921,Non-Expendable
4,4,106,ESY9,11.865480,1699.522299,748.364648,914.222777,38.386337,73.207364,25.048450,...,0.932737,1.971651,4.758553,0.849841,0.625291,21876.446666,1.817433,3.319941,-1.444064,Non-Expendable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,200,194,ÇTL4,66.312500,2707.245283,1763.807018,1644.428571,49.662874,77.694611,25.000000,...,0.301509,-10.835791,3.841059,0.216815,-8.913511,6183.193347,12.081403,-4.464395,23.299640,Non-Expendable
201,201,174,SRY5,43.965753,5844.249147,1282.745946,1593.472028,35.871667,78.000000,25.000000,...,0.252526,-11.829641,4.630384,0.761411,-0.707235,5780.767468,6.449791,-4.664011,9.723321,Non-Expendable
202,202,113,EYP7,57.964912,3125.559006,1609.653226,1435.006211,36.931579,77.485380,25.584795,...,0.281180,-11.248254,4.013784,0.335987,-7.117759,5621.085768,8.406897,-4.743218,14.441382,Non-Expendable
203,203,193,ÇTL3,77.665428,2778.379699,2274.988636,1303.626415,46.983883,76.098901,25.000000,...,0.344372,-9.966112,3.856494,0.227465,-8.753034,5538.420941,9.458481,-4.784222,16.976473,Non-Expendable


In [67]:
data

{'stations': [{'id': '',
   'region': '',
   'importance': '',
   'assignedPersonnel': [{'id': '',
     'name': '',
     'roles': [''],
     'assignedFrom': '',
     'homeStationId': '',
     'negativeStations': [],
     'preferredStations': [],
     'rotationCount': 0}],
   'maxRotationCount': 5}],
 'neighboringRegions': {'Adalar': ['Kartal', 'Pendik'],
  'Arnavutköy': ['Çatalca', 'Esenler', 'Başakşehir', 'Eyüpsultan'],
  'Ataşehir': ['Üsküdar', 'Kadıköy', 'Maltepe', 'Ümraniye'],
  'Avcılar': ['Küçükçekmece', 'Bağcılar', 'Başakşehir'],
  'Bağcılar': ['Küçükçekmece',
   'Bahçelievler',
   'Güngören',
   'Esenler',
   'Avcılar'],
  'Bahçelievler': ['Bağcılar', 'Bakırköy', 'Güngören'],
  'Bakırköy': ['Bahçelievler', 'Güngören', 'Küçükçekmece', 'Esenyurt'],
  'Başakşehir': ['Arnavutköy',
   'Eyüpsultan',
   'Esenler',
   'Avcılar',
   'Küçükçekmece'],
  'Bayrampaşa': ['Gaziosmanpaşa', 'Eyüpsultan', 'Fatih', 'Şişli'],
  'Beşiktaş': ['Şişli', 'Beyoğlu', 'Sarıyer', 'Kağıthane'],
  'Beykoz': 

In [68]:
df_shifts["Görev"].unique()

array(['driver', 'medic'], dtype=object)

In [69]:
def assignments(df_shifts):
    assignment_mask = np.select(
        [
            df_shifts["Görev"] == "Sürücü",
            (df_shifts["Görev"]=="Ekip Sorumlusu") | (df_shifts['Görev']=="Yardımcı Sağlık Personeli")
        ],
        [
            "driver",
            "medic"
        ],
        default=pd.NA
    )
    return assignment_mask

In [70]:
df_shifts['roles']= df_shifts.apply(assignments, axis=1)

C:\Users\mkaya\AppData\Local\Temp\ipykernel_2992\3402541198.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shifts['roles']= df_shifts.apply(assignments, axis=1)


In [71]:
df_shifts

7,İsim,Soyisim,Kimlik No,Mesai,Görev,Branş,Kadro,Telefon,Başlangıç Tarihi,Bitiş Tarihi,Açıklama,İstasyon,İsim Soyisim,region,roles
396,AHMET,AKAR,54001069184,12 Saat,driver,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05468122215,25-06-2025 08:00,25-06-2025 20:00,NaN,ARN1,AHMET AKAR,Arnavutköy,<NA>
402,FEVZİ,BİÇER,22702724848,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05353780732,25-06-2025 08:00,26-06-2025 08:00,NaN,ARN2,FEVZİ BİÇER,Arnavutköy,<NA>
403,HOZAN,EREN,25303629178,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05458503007,25-06-2025 08:00,26-06-2025 08:00,NaN,ARN2,HOZAN EREN,Arnavutköy,<NA>
404,SAMET,SÜME,30670657914,12 Saat,driver,8322.04 AMBULANS SÜRÜCÜSÜ,Sürekli İşçi (696 sayılı KHK),05422820029,25-06-2025 08:00,25-06-2025 20:00,NaN,ARN2,SAMET SÜME,Arnavutköy,<NA>
411,AYŞEANA,KÖSEOĞLU,69445165774,24 Saat,medic,İLK VE ACİL YARDIM,657/4B Sözleşmeli Personel (663 sayılı KHK 45/A),05451052205,25-06-2025 08:00,26-06-2025 08:00,NaN,ARN3,AYŞEANA KÖSEOĞLU,Arnavutköy,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,NESİBE ŞEYDA,ÇIRAK,39352076050,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05352358517,25-06-2025 08:00,26-06-2025 08:00,NaN,ZTB5,NESİBE ŞEYDA ÇIRAK,Zeytinburnu,<NA>
2393,TUĞÇE,ŞAHİN,17362615056,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05445479115,25-06-2025 08:00,26-06-2025 08:00,NaN,ZTB6,TUĞÇE ŞAHİN,Zeytinburnu,<NA>
2394,YUNUS,KILIÇ,64321260598,24 Saat,driver,İLK VE ACİL YARDIM,657/4A Memur,05436687855,25-06-2025 08:00,26-06-2025 08:00,NaN,ZTB6,YUNUS KILIÇ,Zeytinburnu,<NA>
2400,AYŞE,KARTAL,26669574218,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05300530112,25-06-2025 08:00,26-06-2025 08:00,NaN,ZTB7,AYŞE KARTAL,Zeytinburnu,<NA>


In [72]:
df_station_locations

,Ekip Adı,lat,lon,Bölge,city,district,station_count,rural,double_station,double_station_with,Neighbourhood,District
0,ARN1,41.177852,28.745802,ARN,ARNAVUTKÖY,ARNAVUTKÖY MERKEZ,1,KENTSEL,False,NaN,Arnavutköy Merkez Mahallesi,Arnavutköy
1,ARN2,41.156037,28.623824,ARN,ARNAVUTKÖY,HADIMKÖY,1,KENTSEL,False,NaN,Hadımköy Mahallesi,Arnavutköy
2,ARN3,41.204411,28.716732,ARN,ARNAVUTKÖY,MAREŞAL FEVZİ ÇAKMAK,1,KENTSEL,False,NaN,Mareşal Fevzi Çakmak Mahallesi,Arnavutköy
3,ARN4,41.199450,28.760639,ARN,ARNAVUTKÖY,HİCRET,1,KENTSEL,False,NaN,Hicret Mahallesi,Arnavutköy
4,ARN6,41.185858,28.708606,ARN,ARNAVUTKÖY,TAŞOLUK,1,KENTSEL,False,NaN,Taşoluk Mahallesi,Arnavutköy
...,...,...,...,...,...,...,...,...,...,...,...,...
207,ZTB7,41.006756,28.897603,ZTB,ZEYTİNBURNU,SEYİTNİZAM,1,KENTSEL,False,NaN,Seyitnizam Mahallesi,Zeytinburnu
208,ARN5,41.265160,28.688026,ARN,ARNAVUTKÖY,TAYAKADIN,1,KIRSAL,False,NaN,Tayakadın Mahallesi,Arnavutköy
209,ÇTL4,41.413472,28.186711,ÇTL,ÇATALCA,FATİH,1,KIRSAL,False,NaN,Fatih Mahallesi,Çatalca
210,EYP6,41.253788,28.714775,EYP,ARNAVUTKÖY,TAYAKADIN,1,KIRSAL,True,EYP7,Tayakadın Mahallesi,Arnavutköy


In [73]:

station_names= df_station_locations['Ekip Adı'].unique()

In [74]:
data['stations'][0]

{'id': '',
 'region': '',
 'importance': '',
 'assignedPersonnel': [{'id': '',
   'name': '',
   'roles': [''],
   'assignedFrom': '',
   'homeStationId': '',
   'negativeStations': [],
   'preferredStations': [],
   'rotationCount': 0}],
 'maxRotationCount': 5}

In [75]:
df_scorings

,Unnamed: 0,index,Ekip No,total_distance,field_operation_time,hospital_delivery_time,case_response_time,yas_score,cinsiyet_score,yeni_dogan_score,...,case_count_logarithmic_score_MinMax_score,case_count_logarithmic_score_MinMax_score_z_score,teams_population_density_logarithmic_score,teams_population_density_logarithmic_score_MinMax_score,teams_population_density_logarithmic_score_MinMax_score_z_score,total_score,strategic_score,total_score_z,strategic_score_z,station_expendable
0,0,102,ESY5,15.321716,1705.467972,841.280177,891.435094,39.681936,73.595506,25.172861,...,1.000000,3.336389,4.849307,0.912457,1.568817,25105.854797,1.926214,4.921827,-1.181821,Non-Expendable
1,1,101,ESY3,16.187135,1804.872568,722.599260,1014.164519,40.408671,74.646560,25.047125,...,0.978497,2.900107,4.976190,1.000000,2.887967,23669.687726,2.120259,4.209444,-0.714030,Non-Expendable
2,2,99,ESY14,12.193798,1758.070866,777.571702,868.344288,39.532523,75.531915,25.000000,...,0.900457,1.316690,4.720795,0.823790,0.232733,23343.681759,2.404106,4.047735,-0.029750,Non-Expendable
3,3,94,ESY1,16.519174,1657.154448,757.614035,871.449505,37.307183,74.007561,25.000000,...,0.973539,2.799495,4.808144,0.884057,1.140867,22348.994302,1.919536,3.554339,-1.197921,Non-Expendable
4,4,106,ESY9,11.865480,1699.522299,748.364648,914.222777,38.386337,73.207364,25.048450,...,0.932737,1.971651,4.758553,0.849841,0.625291,21876.446666,1.817433,3.319941,-1.444064,Non-Expendable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,200,194,ÇTL4,66.312500,2707.245283,1763.807018,1644.428571,49.662874,77.694611,25.000000,...,0.301509,-10.835791,3.841059,0.216815,-8.913511,6183.193347,12.081403,-4.464395,23.299640,Non-Expendable
201,201,174,SRY5,43.965753,5844.249147,1282.745946,1593.472028,35.871667,78.000000,25.000000,...,0.252526,-11.829641,4.630384,0.761411,-0.707235,5780.767468,6.449791,-4.664011,9.723321,Non-Expendable
202,202,113,EYP7,57.964912,3125.559006,1609.653226,1435.006211,36.931579,77.485380,25.584795,...,0.281180,-11.248254,4.013784,0.335987,-7.117759,5621.085768,8.406897,-4.743218,14.441382,Non-Expendable
203,203,193,ÇTL3,77.665428,2778.379699,2274.988636,1303.626415,46.983883,76.098901,25.000000,...,0.344372,-9.966112,3.856494,0.227465,-8.753034,5538.420941,9.458481,-4.784222,16.976473,Non-Expendable


In [76]:
data

{'stations': [{'id': '',
   'region': '',
   'importance': '',
   'assignedPersonnel': [{'id': '',
     'name': '',
     'roles': [''],
     'assignedFrom': '',
     'homeStationId': '',
     'negativeStations': [],
     'preferredStations': [],
     'rotationCount': 0}],
   'maxRotationCount': 5}],
 'neighboringRegions': {'Adalar': ['Kartal', 'Pendik'],
  'Arnavutköy': ['Çatalca', 'Esenler', 'Başakşehir', 'Eyüpsultan'],
  'Ataşehir': ['Üsküdar', 'Kadıköy', 'Maltepe', 'Ümraniye'],
  'Avcılar': ['Küçükçekmece', 'Bağcılar', 'Başakşehir'],
  'Bağcılar': ['Küçükçekmece',
   'Bahçelievler',
   'Güngören',
   'Esenler',
   'Avcılar'],
  'Bahçelievler': ['Bağcılar', 'Bakırköy', 'Güngören'],
  'Bakırköy': ['Bahçelievler', 'Güngören', 'Küçükçekmece', 'Esenyurt'],
  'Başakşehir': ['Arnavutköy',
   'Eyüpsultan',
   'Esenler',
   'Avcılar',
   'Küçükçekmece'],
  'Bayrampaşa': ['Gaziosmanpaşa', 'Eyüpsultan', 'Fatih', 'Şişli'],
  'Beşiktaş': ['Şişli', 'Beyoğlu', 'Sarıyer', 'Kağıthane'],
  'Beykoz': 

In [77]:
station_names[1]

'ARN2'

In [78]:
df_shifts

7,İsim,Soyisim,Kimlik No,Mesai,Görev,Branş,Kadro,Telefon,Başlangıç Tarihi,Bitiş Tarihi,Açıklama,İstasyon,İsim Soyisim,region,roles
396,AHMET,AKAR,54001069184,12 Saat,driver,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05468122215,25-06-2025 08:00,25-06-2025 20:00,NaN,ARN1,AHMET AKAR,Arnavutköy,<NA>
402,FEVZİ,BİÇER,22702724848,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05353780732,25-06-2025 08:00,26-06-2025 08:00,NaN,ARN2,FEVZİ BİÇER,Arnavutköy,<NA>
403,HOZAN,EREN,25303629178,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05458503007,25-06-2025 08:00,26-06-2025 08:00,NaN,ARN2,HOZAN EREN,Arnavutköy,<NA>
404,SAMET,SÜME,30670657914,12 Saat,driver,8322.04 AMBULANS SÜRÜCÜSÜ,Sürekli İşçi (696 sayılı KHK),05422820029,25-06-2025 08:00,25-06-2025 20:00,NaN,ARN2,SAMET SÜME,Arnavutköy,<NA>
411,AYŞEANA,KÖSEOĞLU,69445165774,24 Saat,medic,İLK VE ACİL YARDIM,657/4B Sözleşmeli Personel (663 sayılı KHK 45/A),05451052205,25-06-2025 08:00,26-06-2025 08:00,NaN,ARN3,AYŞEANA KÖSEOĞLU,Arnavutköy,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,NESİBE ŞEYDA,ÇIRAK,39352076050,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05352358517,25-06-2025 08:00,26-06-2025 08:00,NaN,ZTB5,NESİBE ŞEYDA ÇIRAK,Zeytinburnu,<NA>
2393,TUĞÇE,ŞAHİN,17362615056,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05445479115,25-06-2025 08:00,26-06-2025 08:00,NaN,ZTB6,TUĞÇE ŞAHİN,Zeytinburnu,<NA>
2394,YUNUS,KILIÇ,64321260598,24 Saat,driver,İLK VE ACİL YARDIM,657/4A Memur,05436687855,25-06-2025 08:00,26-06-2025 08:00,NaN,ZTB6,YUNUS KILIÇ,Zeytinburnu,<NA>
2400,AYŞE,KARTAL,26669574218,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05300530112,25-06-2025 08:00,26-06-2025 08:00,NaN,ZTB7,AYŞE KARTAL,Zeytinburnu,<NA>


In [87]:
def dummy_func(row):
    return row

In [90]:
df_shifts.apply(dummy_func, axis=1)

7,İsim,Soyisim,Kimlik No,Mesai,Görev,Branş,Kadro,Telefon,Başlangıç Tarihi,Bitiş Tarihi,Açıklama,İstasyon,İsim Soyisim,region,roles
396,AHMET,AKAR,54001069184,12 Saat,driver,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05468122215,25-06-2025 08:00,25-06-2025 20:00,NaN,ARN1,AHMET AKAR,Arnavutköy,<NA>
402,FEVZİ,BİÇER,22702724848,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05353780732,25-06-2025 08:00,26-06-2025 08:00,NaN,ARN2,FEVZİ BİÇER,Arnavutköy,<NA>
403,HOZAN,EREN,25303629178,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05458503007,25-06-2025 08:00,26-06-2025 08:00,NaN,ARN2,HOZAN EREN,Arnavutköy,<NA>
404,SAMET,SÜME,30670657914,12 Saat,driver,8322.04 AMBULANS SÜRÜCÜSÜ,Sürekli İşçi (696 sayılı KHK),05422820029,25-06-2025 08:00,25-06-2025 20:00,NaN,ARN2,SAMET SÜME,Arnavutköy,<NA>
411,AYŞEANA,KÖSEOĞLU,69445165774,24 Saat,medic,İLK VE ACİL YARDIM,657/4B Sözleşmeli Personel (663 sayılı KHK 45/A),05451052205,25-06-2025 08:00,26-06-2025 08:00,NaN,ARN3,AYŞEANA KÖSEOĞLU,Arnavutköy,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,NESİBE ŞEYDA,ÇIRAK,39352076050,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05352358517,25-06-2025 08:00,26-06-2025 08:00,NaN,ZTB5,NESİBE ŞEYDA ÇIRAK,Zeytinburnu,<NA>
2393,TUĞÇE,ŞAHİN,17362615056,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05445479115,25-06-2025 08:00,26-06-2025 08:00,NaN,ZTB6,TUĞÇE ŞAHİN,Zeytinburnu,<NA>
2394,YUNUS,KILIÇ,64321260598,24 Saat,driver,İLK VE ACİL YARDIM,657/4A Memur,05436687855,25-06-2025 08:00,26-06-2025 08:00,NaN,ZTB6,YUNUS KILIÇ,Zeytinburnu,<NA>
2400,AYŞE,KARTAL,26669574218,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05300530112,25-06-2025 08:00,26-06-2025 08:00,NaN,ZTB7,AYŞE KARTAL,Zeytinburnu,<NA>


In [79]:
df_shifts['Kimlik No']= df_shifts['Kimlik No'].astype('int64')

C:\Users\mkaya\AppData\Local\Temp\ipykernel_2992\3537363259.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shifts['Kimlik No']= df_shifts['Kimlik No'].astype('int64')


In [80]:
json_input = '''
    {
        "stations": [
            {
                "id": "",
                "region": "",
                "importance": "",
                "assignedPersonnel": [
                    {"id": "", "name": "", "roles": [""], "assignedFrom": "", "homeStationId": "", "negativeStations": [], "preferredStations": [], "rotationCount": 0}
                ],
                "maxRotationCount": 5
            }
        ],
        "neighboringRegions" : {
            "Adalar": ["Kartal", "Pendik"],
            "Arnavutköy": ["Çatalca", "Esenler", "Başakşehir", "Eyüpsultan"],
            "Ataşehir": ["Üsküdar", "Kadıköy", "Maltepe", "Ümraniye"],
            "Avcılar": ["Küçükçekmece", "Bağcılar", "Başakşehir"],
            "Bağcılar": ["Küçükçekmece", "Bahçelievler", "Güngören", "Esenler", "Avcılar"],
            "Bahçelievler": ["Bağcılar", "Bakırköy", "Güngören"],
            "Bakırköy": ["Bahçelievler", "Güngören", "Küçükçekmece", "Esenyurt"],
            "Başakşehir": ["Arnavutköy", "Eyüpsultan", "Esenler", "Avcılar", "Küçükçekmece"],
            "Bayrampaşa": ["Gaziosmanpaşa", "Eyüpsultan", "Fatih", "Şişli"],
            "Beşiktaş": ["Şişli", "Beyoğlu", "Sarıyer", "Kağıthane"],
            "Beykoz": ["Ümraniye", "Çekmeköy", "Sancaktepe", "Üsküdar"],
            "Beylikdüzü": ["Büyükçekmece", "Esenyurt", "Avcılar"],
            "Beyoğlu": ["Şişli", "Kağıthane", "Fatih", "Beşiktaş", "Eyüpsultan"],
            "Büyükçekmece": ["Çatalca", "Silivri", "Esenyurt", "Beylikdüzü"],
            "Çatalca": ["Silivri", "Büyükçekmece", "Arnavutköy"],
            "Çekmeköy": ["Ümraniye", "Beykoz", "Üsküdar", "Sancaktepe"],
            "Esenler": ["Bağcılar", "Güngören", "Bakırköy", "Başakşehir", "Gaziosmanpaşa"],
            "Esenyurt": ["Küçükçekmece", "Avcılar", "Bakırköy", "Beylikdüzü", "Büyükçekmece"],
            "Eyüpsultan": ["Sarıyer", "Kağıthane", "Beyoğlu", "Gaziosmanpaşa", "Bayrampaşa", "Fatih", "Sultangazi", "Başakşehir", "Arnavutköy"],
            "Fatih": ["Beyoğlu", "Şişli", "Eminönü", "Esenler"],
            "Gaziosmanpaşa": ["Eyüpsultan", "Sultangazi", "Esenler", "Bayrampaşa"],
            "Güngören": ["Bağcılar", "Bahçelievler", "Bakırköy", "Esenler", "Zeytinburnu"],
            "Kadıköy": ["Ümraniye", "Ataşehir", "Maltepe", "Kartal", "Sancaktepe"],
            "Kağıthane": ["Şişli", "Beşiktaş", "Beyoğlu", "Eyüpsultan", "Sarıyer", "Sultangazi"],
            "Kartal": ["Maltepe", "Pendik", "Tuzla", "Adalar"],
            "Küçükçekmece": ["Avcılar", "Bağcılar", "Bakırköy", "Esenyurt", "Başakşehir"],
            "Maltepe": ["Kadıköy", "Kartal", "Pendik", "Sancaktepe", "Ümraniye"],
            "Pendik": ["Kartal", "Tuzla", "Sancaktepe", "Maltepe", "Ümraniye"],
            "Sancaktepe": ["Çekmeköy", "Ümraniye", "Kadıköy", "Maltepe", "Pendik", "Tuzla", "Kartal"],
            "Sarıyer": ["Eyüpsultan", "Kağıthane", "Beşiktaş", "Beykoz", "Şile"],
            "Silivri": ["Çatalca", "Büyükçekmece"],
            "Sultanbeyli": ["Pendik", "Kartal", "Sancaktepe", "Ümraniye"],
            "Sultangazi": ["Eyüpsultan", "Gaziosmanpaşa", "Kağıthane"],
            "Şile": ["Sarıyer"],
            "Şişli": ["Beyoğlu", "Beşiktaş", "Kağıthane", "Bayrampaşa", "Fatih"],
            "Tuzla": ["Pendik", "Kartal"],
            "Ümraniye": ["Üsküdar","Ataşehir", "Kadıköy", "Maltepe", "Sancaktepe", "Çekmeköy",  "Beykoz"],
            "Üsküdar": ["Ümraniye", "Çekmeköy", "Beykoz", "Kadıköy", "Ataşehir"],
            "Zeytinburnu": ["Bakırköy", "Güngören", "Fatih", "Bayrampaşa"]
        },
        "maxRotationsPerPersonnel": 5
    }
'''

In [81]:
data = json.loads(json_input)

In [ ]:
for station_name in station_names:
    df_assignments = df_shifts[df_shifts['İstasyon'] == station_name].copy()

    data['stations'][-1]['id'] = station_name

    # Get region as string
    region_series = df_station_locations[df_station_locations['Ekip Adı'] == station_name]['District']
    data['stations'][-1]['region'] = region_series.iloc[0] if not region_series.empty else ""

    # Get importance as float (fix here!)
    importance_series = df_scorings[df_scorings['Ekip No'] == station_name]['total_score_z']
    data['stations'][-1]['importance'] = float(importance_series.iloc[0]) if not importance_series.empty else 0.0
    
    added_worker_ids= []
    
    for i in range(len(df_assignments)):
        worker_id= df_assignments.iloc[i]['Kimlik No']
        if worker_id  not in added_worker_ids:
            added_worker_ids.append(df_assignments.iloc[i]['Kimlik No'])
            data['stations'][-1]['assignedPersonnel'][-1]['id'] = str(worker_id)
            data['stations'][-1]['assignedPersonnel'][-1]['name'] = str(df_assignments.iloc[i]['İsim Soyisim'])
            data['stations'][-1]['assignedPersonnel'][-1]['roles'][0] = str(df_assignments.iloc[i]['Görev'])
            data['stations'][-1]['assignedPersonnel'][-1]['homeStationId'] = str(df_assignments.iloc[i]['İstasyon'])
            data['stations'][-1]['assignedPersonnel'][-1]['assignedFrom']= str(df_assignments.iloc[i]['İstasyon'])
        else:
            print(f"Worker id {worker_id} is already assigned in another station")
            pass
            
        if i != len(df_assignments) - 1:            
            data['stations'][-1]['assignedPersonnel'].append({
                'id': '',
                'name': '',
                'roles': [''],
                'assignedFrom': '',
                'homeStationId': '',
                'negativeStations': [],
                'preferredStations': [],
                'rotationCount': 0})
        else:
            pass
    if station_name != station_names[-1]:      
        data['stations'].append({
        'id': '',
        'region': '',
        'importance': '',
        'assignedPersonnel': [{'id': '',
        'name': '',
        'roles': [''],
        'assignedFrom': '',
        'homeStationId': '',
        'negativeStations': [],
        'preferredStations': [],
        'rotationCount': 0}],
        'maxRotationCount': 5})
    else:
        pass

In [83]:
with open('C:/Users/mkaya/Onedrive/Masaüstü/data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [84]:
with open('C:/Users/mkaya/Onedrive/Masaüstü/data.json', 'r', encoding='utf-8') as f:
    loaded_data = json.load(f)

In [85]:
loaded_data

{'stations': [{'id': 'ARN1',
   'region': 'Arnavutköy',
   'importance': 0.7192875966748681,
   'assignedPersonnel': [{'id': '54001069184',
     'name': 'AHMET AKAR',
     'roles': ['driver'],
     'assignedFrom': 'ARN1',
     'homeStationId': 'ARN1',
     'negativeStations': [],
     'preferredStations': [],
     'rotationCount': 0}],
   'maxRotationCount': 5},
  {'id': 'ARN2',
   'region': 'Arnavutköy',
   'importance': -2.019217763845094,
   'assignedPersonnel': [{'id': '22702724848',
     'name': 'FEVZİ BİÇER',
     'roles': ['medic'],
     'assignedFrom': 'ARN2',
     'homeStationId': 'ARN2',
     'negativeStations': [],
     'preferredStations': [],
     'rotationCount': 0},
    {'id': '25303629178',
     'name': 'HOZAN EREN',
     'roles': ['medic'],
     'assignedFrom': 'ARN2',
     'homeStationId': 'ARN2',
     'negativeStations': [],
     'preferredStations': [],
     'rotationCount': 0},
    {'id': '30670657914',
     'name': 'SAMET SÜME',
     'roles': ['driver'],
     'ass